### Consulta 1
def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

-- precio en tabla steam_games
-- recommend (True or false) en tabla user_revs
-- cantidad de items sería un count de unique (item_id) en tabla user_items

In [48]:
import pandas as pd

In [49]:
columns = ['id', 'price']
games_price = pd.read_csv(r'./datasets/steam_games.csv', usecols=columns)
games_price.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [50]:
games_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [51]:
unique_list = []
for i in games_price['price']:
    try:
        float(i)
    except:
        unique_list.append(i)

uniques = set(unique_list)
uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']

In [52]:
print(uniques)

{'Play WARMACHINE: Tactics Demo', 'Free To Play', 'Free to Play', 'Install Now', 'Free HITMAN™ Holiday Pack', 'Install Theme', 'Starting at $499.00', 'Free to Try', 'Free Movie', 'Starting at $449.00', 'Play for Free!', 'Third-party', 'Play Now', 'Free Demo', 'Play the Demo', 'Free Mod', 'Free', 'Free to Use'}


In [53]:
games_price['price_fixed'] = games_price['price'].apply(lambda x: 0 if x in uniques 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
games_price.head()

,price,id,price_fixed
0,4.99,761140.0,4.99
1,Free To Play,643980.0,0
2,Free to Play,670290.0,0
3,0.99,767400.0,0.99
4,2.99,773570.0,2.99


In [54]:
games_price['price_fixed'] = games_price['price_fixed'].astype(float)

In [55]:
games_price.rename(columns={'id':'item_id'},inplace=True)
games_price.drop(columns='price', inplace=True)
games_price.head()

,item_id,price_fixed
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


In [56]:
games_price.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [57]:
games_price.fillna(0, inplace=True)

In [58]:
games_price.isna().sum()

item_id        0
price_fixed    0
dtype: int64

In [60]:
columns = ['user_id', 'item_id', 'items_count']
user_items = pd.read_csv('datasets/aus_users_items.csv', usecols=columns)
user_items.head()

,user_id,items_count,item_id
0,76561197970982479,277,10
1,76561197970982479,277,20
2,76561197970982479,277,30
3,76561197970982479,277,40
4,76561197970982479,277,50


In [61]:
df_merged = user_items.merge(games_price, on='item_id', how='left')
df_merged.sample(10)

,user_id,items_count,item_id,price_fixed
2758822,76561198078212818,372,346010,7.99
4660211,corporalecho,140,367690,NaN
1169962,jet14,110,239030,9.99
3917914,76561197990835550,215,55110,19.99
4299998,tommmmmmmmm,43,219540,NaN
4455716,76561198052032850,40,380,7.99
4736084,76561198069478280,11,304270,NaN
3573051,76561198091638676,29,440090,0.00
2391393,76561198057674714,71,50,4.99
4219997,76561198037336064,76,202090,NaN


In [86]:
user_spent = df_merged.groupby('user_id')['price_fixed'].agg('sum')
display(user_spent)

user_id
--000--               402.77
--ace--               166.82
--ionex--             109.92
-2SV-vuLB-Kg          437.49
-404PageNotFound-    1514.31
                      ...   
zzonci                 19.98
zzoptimuszz            64.98
zzydrax                99.94
zzyfo                 828.51
zzzmidmiss            453.67
Name: price_fixed, Length: 70912, dtype: float64

In [63]:
columns = ['user_id', 'recommend']
user_revs = pd.read_csv('./datasets/aus_user_revs_clean.csv', usecols=columns)



In [78]:
user_revs.sample(5)

,user_id,recommend
267,76561198126640317,True
1700,mainkradxdxdx,True
36087,lollipopwarrior,True
32638,76561198035590106,True
17020,POKESTEAM,True


In [85]:
user_revs.groupby('user_id')['recommend'].agg('count')

user_id
--000--         1
--ace--         2
--ionex--       2
-2SV-vuLB-Kg    5
-Azsael-        1
               ..
zwanzigdrei     1
zy0705          1
zynxgameth      1
zyr0n1c         9
zzoptimuszz     1
Name: recommend, Length: 25458, dtype: int64

In [81]:
# df_merged_final = user_items.merge(user_revs, on='user_id', how='left')


In [83]:
# df_merged_final.sample(20)

,user_id,items_count,item_id,recommend
6127825,cod1245678,135,202070,NaN
3658413,sterscram,93,56400,True
1924515,CoNnEcT500BR,136,202090,True
8713778,76561198026996483,797,386880,True
3095168,76561198012040937,50,23310,NaN
5595717,76561198133624222,186,409710,True
2121867,randyy242,514,305620,True
3527295,z3b1z,300,255520,True
7211681,76561198044448119,52,239220,NaN
9190367,rezmwashere,89,252490,True
